In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc

### SIR with numpy
square lattice

In [2]:
#parameters
p_t = 0.6
p_r = 0.3
S = 0
I = 1
R = 2
L = 5
N = L * L
T = np.zeros(N,dtype=int)
T[0] = I
T[1] = I
net = np.zeros((N,4),dtype = int)
for i in range(N):
    x = i%L
    y = i//L
    net[i][0] = (x+1)%L + y*L
    net[i][1] = (x-1)%L + y*L
    net[i][2] = x%L + ((y+1)%L)*L
    net[i][3] = x%L + ((y-1)%L)*L

In [3]:
# The net array contains the identifiers of the neighbors, e.g. net[8]
# left, right: 7, 9
# down, up: (+-L) 3, 13
# e.g. net[0] (top left corner)
# right: 1
# left: the rightmost in the row because of the periodic boundary condition, L-1=4
# down: 5 = 0+L
# up: (leftmost in the bottom row) 20 = (L-1)*5
net[8], net[0]

(array([ 9,  7, 13,  3]), array([ 1,  4,  5, 20]))

In [4]:
# The array with the indices of the infected nodes
infected = np.where(T == I)[0]
infected

array([0, 1])

In [5]:
# The coordinates of the neighbors of the infected ones
infected_neighs = net[infected]
infected_neighs

array([[ 1,  4,  5, 20],
       [ 2,  0,  6, 21]])

In [6]:
# The state of the neighbors (0: S, 1:I)
T[net[infected]]

array([[1, 0, 0, 0],
       [0, 1, 0, 0]])

In [7]:
# Now we create a mask that will be True where S gets infected. 
# This consists of two parts: first, is it an S?

In [8]:
T[net[infected]] == S

array([[False,  True,  True,  True],
       [ True, False,  True,  True]])

In [9]:
# And second, whether the random number drawn is smaller than p.  
# We generate the random number array with the same shape as before.
p_t > np.random.random(size=infected_neighs.shape)

array([[False,  True, False,  True],
       [ True,  True,  True,  True]])

In [10]:
# Reminder (1: True, 0: False):
print(True*True, True*False, False*False)

1 0 0


In [11]:
# In other words, the element-wise product of the two above will be True (i.e. 1) wherever both were True.
(T[net[infected]] == S) * (p_t > np.random.random(size=infected_neighs.shape))

array([[False,  True,  True, False],
       [False, False,  True,  True]])

In [12]:
# Finally we have to change the state of the nodes for which the above is True:
r = (T[net[infected]] == S) * (p_t > np.random.random(size=infected_neighs.shape))
print(r)
print(T[infected_neighs])
T[infected_neighs[r]] = I
print(T[infected_neighs])

[[False False  True False]
 [ True False  True  True]]
[[1 0 0 0]
 [0 1 0 0]]
[[1 0 1 0]
 [1 1 1 1]]


In [13]:
# Recovery is easier we change the state of those infected ones for which we roll 
# sufficiently small random number
print(infected)
r = np.random.random(size=infected.shape) < p_r
print(r)
T[infected[r]] = R
print(T[infected])

[0 1]
[ True False]
[2 1]


### Put together

In [14]:
replay = []
p_t = 0.6
p_r = 0.3
S = 0
I = 1
R = 2
L = 100
N = L * L
T = np.zeros(N,dtype=int)
T[0] = I
T[1] = I
net = np.zeros((N,4),dtype = int)
for i in range(N):
    x = i%L
    y = i//L
    net[i][0] = (x+1)%L + y*L
    net[i][1] = (x-1)%L + y*L
    net[i][2] = x%L + ((y+1)%L)*L
    net[i][3] = x%L + ((y-1)%L)*L
for t in range(200):
    infected = np.where(T == I)[0]
    if len(infected) == 0:
        break
    infected_neighs = net[infected]
    T[infected_neighs[(T[infected_neighs] == S) * p_t > np.random.random(size=infected_neighs.shape)]] = I
    T[infected[np.random.random(size=infected.shape) < p_r]] = R
    replay.append(np.copy(T.reshape(L,L)))

In [15]:
%%capture
fig = plt.figure()
ims = []
for s in replay:
    im = plt.imshow(s,animated=True,vmin=0,vmax=2,cmap='rainbow')
    plt.axis('off')
    ims.append([im])
ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True)
rc('animation', html='jshtml')


In [16]:
ani

## Networks
 * the `net` array should be as wide as the largest degree.
 * excess neighbors in the `net` array connect to node `0`
 * node `0` is not part of the network and should be set to recovered

## Tasks
We define the following quantities:
 * $P_\infty$ the probability that a node belongs to the giant component. This is the size of the largest component divided by the number of nodes
 * $S$ the average size of the components without the largest one

Start from the following networks:
 * ER, with $\langle k \rangle = 2$, $N=1000$
 * BE, with $m=2$, $N=1000$
 * the euflight data

Choose at least 4 different centrality measures (degree and betweennes should be included) measure and plot $P_\infty$ and $S$ as fucntion of the fraction of nodes removed. Remove nodes by ascending and descending centrality order. 


In [18]:
import networkx as nx
# Parameters
N = 1000  # Number of nodes
k_avg = 2  # Average degree
p = k_avg / (N - 1)  # Connection probability

# Create ER network
G = nx.erdos_renyi_graph(N, p)

# Verify the average degree
actual_avg_degree = sum(dict(G.degree()).values()) / N
print(f"Target average degree: {k_avg}")
print(f"Actual average degree: {actual_avg_degree:.3f}")
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

Target average degree: 2
Actual average degree: 2.014
Number of nodes: 1000
Number of edges: 1007
